In [1]:
import fastbook
fastbook.setup_book()
print("ok")

ok


In [2]:
from fastai.vision.all import *
from fastbook import *
print("ok")

ok


In [14]:
print(URLs.MNIST)
untar_data(URLs.MNIST, base='~/src/digits/', data='~/src/digits/data')

https://s3.amazonaws.com/fast-ai-imageclas/mnist_png.tgz


Path('/home/john/.fastai/~/src/digits/data/mnist_png')

In [ ]:
base="~/src/digits/data/")

In [10]:
# !pip install nbdev